In [36]:
def BS_model(stock_price, var, strike_price, r_f, ttm):
    import scipy.stats
    import numpy as np
    import math
    norm_dist = scipy.stats.norm(0, 1)
    d1 = 1/(var * math.sqrt(ttm)) * (np.log(stock_price/strike_price) +(r_f + (var**2/2))*ttm)
    d2 = d1 - var*math.sqrt(ttm)
    
    call_value = norm_dist.cdf(d1) * stock_price - norm_dist.cdf(d2) * strike_price * math.exp(-r_f * ttm)
    put_value = strike_price * math.exp(-r_f * ttm) - stock_price + call_value
    
    return call_value, put_value

In [39]:
stock_price = 100
var = 0.1
strike_price = 100
r_f = 0.03
ttm = 1

BS_model(stock_price, var, strike_price, r_f, ttm)

(5.58187715093873, 2.62643050578955)